In [ ]:
import pandas as pd
import numpy as np
import operator
import gzip
import networkx as nx
import time
import random
random.seed(10)

from node2vec import Node2Vec

import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F

import torch_geometric
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, Linear, to_hetero

from sklearn.metrics import precision_score, recall_score, f1_score

from src.utils import *
from src.gnn import *

In [ ]:
print(torch.__version__)

In [ ]:
print(torch_geometric.__version__)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# CaLiGraph

In [ ]:
CLG_dbs = [{'path' : './datasets/clg/clg_10e4/',
            'train_file' : 'clg_10e4-train.nt',
            'test_file' : 'clg_10e4-test.nt'},
           {'path' : './datasets/clg/clg_10e5/',
            'train_file' : 'clg_10e5-train.nt',
            'test_file' : 'clg_10e5-test.nt'}]

In [ ]:
db_ = CLG_dbs[0]
path = db_['path']
train_file = db_['train_file']
test_file = db_['test_file']

print('Running...', train_file, test_file)

df_train = load_clg_files(path+train_file)
g_train, nodes_train, edges_train = create_graph(df_train)
print(f'# Train - Triplets: {len(df_train)}, # Nodes: {g_train.number_of_nodes()}, # Edges: {g_train.number_of_edges()}')
df_test = load_clg_files(path+test_file)
g_test, nodes_test, edges_test = create_graph(df_test)
print(f'# Test - Triplets: {len(df_test)}, # Nodes: {g_test.number_of_nodes()}, # Edges: {g_test.number_of_edges()}')
print()

st = time.time()
model = GNN()
model._train(g_train, 'GAT')
model._eval(g_test, min(g_test.number_of_nodes(),100), 'GAT')
et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')
print()

st = time.time()
model = GNN()
model._train(g_train, 'GAT_2hops')
model._eval(g_test, min(g_test.number_of_nodes(),100), 'GAT_2hops')
et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')
print()